In [1]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRunSup.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
]

values = [1, 2, 3]

df["fraction"] = np.select(conditions, values)


dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0"])
dfPrepSample.head()


,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
0,66,25310,2625,24.0,149.0,808,1
1,46,28154,5825,96.0,174.0,836,2
2,45,7361,1400,9.0,74.0,390,3
3,34,8488,2125,26.5,74.0,390,3
4,27,4589,550,6.0,46.0,261,1


In [2]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
517,67,22263,1700,30.0,133.0,660,2
2788,74,88303,4610,44.0,196.0,1094,1
1655,66,35571,6575,105.0,196.0,945,2
3216,66,23454,3000,34.0,173.0,706,1
2719,73,40865,9400,103.0,198.0,791,3


In [3]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
2536,40,6691,1400,11.0,60.0,366,3
1633,12,70,100,0.0,14.0,15,3
654,57,13434,2725,30.5,98.0,525,3
667,68,25812,1250,11.0,149.0,682,1
2613,27,5221,1750,17.0,58.0,343,3


In [4]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
517,67,22263,1700,30.0,133.0,660,2
2788,74,88303,4610,44.0,196.0,1094,1
1655,66,35571,6575,105.0,196.0,945,2
3216,66,23454,3000,34.0,173.0,706,1
2719,73,40865,9400,103.0,198.0,791,3
...,...,...,...,...,...,...,...
341,78,19630,3550,28.0,138.0,614,3
1685,77,44673,15575,125.5,206.0,950,3
987,66,42052,2950,30.0,157.0,1050,1
3098,71,32177,2700,30.0,180.0,774,1


In [5]:

dfTrainLabels = dfTrainFeatures.pop("total_army_value")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[   67., 22263.,    30.,   133.,   660.,     2.],
       [   74., 88303.,    44.,   196.,  1094.,     1.],
       [   66., 35571.,   105.,   196.,   945.,     2.],
       ...,
       [   66., 42052.,    30.,   157.,  1050.,     1.],
       [   71., 32177.,    30.,   180.,   774.,     1.],
       [   50.,  7955.,    18.,    93.,   481.,     1.]])

In [6]:
# normalize = layers.Normalization()
# normalize.adapt(dfTrainFeatures)

In [7]:
sc_model = tf.keras.Sequential([layers.Dense(10, activation="relu"),layers.Dense(3, activation="relu"), layers.Dense(1)]) # letzte lösung 30/5

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [8]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=350)

Epoch 1/350
84/84 [==============================] - 1s 795us/step - loss: 10231860.0000
Epoch 2/350
84/84 [==============================] - 0s 627us/step - loss: 9074564.0000
Epoch 3/350
84/84 [==============================] - 0s 602us/step - loss: 8243496.0000
Epoch 4/350
84/84 [==============================] - 0s 603us/step - loss: 7902138.0000
Epoch 5/350
84/84 [==============================] - 0s 578us/step - loss: 7422519.0000
Epoch 6/350
84/84 [==============================] - 0s 591us/step - loss: 7198350.5000
Epoch 7/350
84/84 [==============================] - 0s 590us/step - loss: 7120580.5000
Epoch 8/350
84/84 [==============================] - 0s 590us/step - loss: 6819107.5000
Epoch 9/350
84/84 [==============================] - 0s 603us/step - loss: 6288183.0000
Epoch 10/350
84/84 [==============================] - 0s 580us/step - loss: 6439908.5000
Epoch 11/350
84/84 [==============================] - 0s 578us/step - loss: 6085102.5000
Epoch 12/350
84/84 [=========

In [9]:
dfTestFeatures = dfTestSample.copy()

dfTestFeatures

,total_workers,resource_mining,total_army_value,total_army,supply,time,fraction
2536,40,6691,1400,11.0,60.0,366,3
1633,12,70,100,0.0,14.0,15,3
654,57,13434,2725,30.5,98.0,525,3
667,68,25812,1250,11.0,149.0,682,1
2613,27,5221,1750,17.0,58.0,343,3
...,...,...,...,...,...,...,...
2432,54,31287,2350,23.0,117.0,858,1
278,52,15077,2650,26.0,106.0,551,3
432,66,25591,3650,53.0,133.0,700,2
3313,63,20543,6350,85.0,152.0,760,3


In [10]:

dfTestLabels = dfTestFeatures.pop("total_army_value")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

2536    1400
1633     100
654     2725
667     1250
2613    1750
Name: total_army_value, dtype: int64

In [11]:
from sklearn.metrics import r2_score

predictedValues = sc_model.predict(dfTestFeatures)

21/21 [==============================] - 0s 543us/step


In [12]:

print(f"The R^2 value in the test set is {r2_score(dfTestLabels, predictedValues)}")

The R^2 value in the test set is 0.7101831765091


In [13]:
# optimizer with scaling and lin reg

import warnings
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr

data = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')
data = data[1350:]


# ,player,total_workers,total_army_value,total_army,fraction,winner,replay_filename
dfOpt = dfPrepSample.drop(columns=[
                               "total_workers"])

dfOpt.head

<bound method NDFrame.head of       resource_mining  total_army_value  total_army  supply  time  fraction
0               25310              2625        24.0   149.0   808         1
1               28154              5825        96.0   174.0   836         2
2                7361              1400         9.0    74.0   390         3
3                8488              2125        26.5    74.0   390         3
4                4589               550         6.0    46.0   261         1
...               ...               ...         ...     ...   ...       ...
3323             7872              1750        25.0    70.0   401         2
3324             6490               600         8.0    39.0   261         1
3325             4774               600         8.0    39.0   251         1
3326            13131               475         5.0    70.0   429         1
3327             9132              3625        35.5    90.0   458         3

[3328 rows x 6 columns]>

In [14]:


# create variables
# create Model

m = gp.Model("Worker optimizer")
gppd.set_interactive()

w = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_workers", vtype=gp.GRB.INTEGER)
av = gppd.add_vars(m, pandas_obj=dfOpt,
                      name="total_army_value")

a = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_army", vtype=gp.GRB.INTEGER)
s = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="supply", vtype=gp.GRB.INTEGER)


m.update()
dfOpt = dfOpt.drop(columns=["total_army_value"])

feats = dfOpt
feats = pd.concat([w, feats], axis=1)
feats.head

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-17


<bound method NDFrame.head of                          total_workers  resource_mining  total_army  supply  \
0        <gurobi.Var total_workers[0]>            25310        24.0   149.0   
1        <gurobi.Var total_workers[1]>            28154        96.0   174.0   
2        <gurobi.Var total_workers[2]>             7361         9.0    74.0   
3        <gurobi.Var total_workers[3]>             8488        26.5    74.0   
4        <gurobi.Var total_workers[4]>             4589         6.0    46.0   
...                                ...              ...         ...     ...   
3323  <gurobi.Var total_workers[3323]>             7872        25.0    70.0   
3324  <gurobi.Var total_workers[3324]>             6490         8.0    39.0   
3325  <gurobi.Var total_workers[3325]>             4774         8.0    39.0   
3326  <gurobi.Var total_workers[3326]>            13131         5.0    70.0   
3327  <gurobi.Var total_workers[3327]>             9132        35.5    90.0   

      time  fraction 

In [15]:


m.setObjective(av.sum(), gp.GRB.MAXIMIZE) # Zielfunktion
"""
gppd.add_constrs(m, ((av - 50*w)/100), gp.GRB.GREATER_EQUAL, a) # Gleichung 2
"""
gppd.add_constrs(m, w + a, gp.GRB.GREATER_EQUAL, 0) # Gleichung 3
gppd.add_constrs(m, w + a, gp.GRB.LESS_EQUAL, s) # Gleichung 4
m.update()

pred_constr = add_predictor_constr(m, sc_model, feats, av) # Einfügen des Regressiosmodells
pred_constr.print_stats()
m.optimize()


Model for sequential1:
249600 variables
119808 constraints
116480 general constraints
Input has shape (3328, 6)
Output has shape (3328, 1)

--------------------------------------------------------------------------------
Layer           Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
dense0            (3328, 30)       199680        99840            0        99840 (relu)

dense1             (3328, 5)        33280        16640            0        16640 (relu)

dense2             (3328, 1)            0         3328            0            0

--------------------------------------------------------------------------------
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 126464 rows, 262912

In [ ]:

outputs = pred_constr.input_values
pred_constr.input_values



GurobiError: Unable to retrieve attribute 'X'